In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%cd '/content/drive/My Drive/Colab Notebooks/0-tratamiento-correcto-columnas'

/content/drive/My Drive/Colab Notebooks/0-tratamiento-correcto-columnas


In [4]:
import os
import random as rd
import numpy as np
import pandas as pd

In [5]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [6]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['data-ct-arroz.csv', 'data-ct-maiz.csv', 'data-cp-cacao-ramilla.csv']

In [7]:
ruta_cacao = os.path.join(ruta_iniciales, 'data-cp-cacao-ramilla.csv')
cacao = pd.read_csv(ruta_cacao)
cacao

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,tratamiento_compra_insumos,tratamiento_ventas_transporte,tratamiento_ventas_demanda,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,...,cp_cantidad_ins_pq,cp_umed_ins_pq,cp_color_ins_pq,cp_cantidad_fun_pq,cp_umed_fun_pq,cp_color_fun_pq,cp_cantidad_pq,cp_umed_pq,cp_color_pq,cp_trabocacont,cp_trabfami,cp_obsotroscp,cp_nclavr,cp_trs,cp_trc,cp_tro,cp_tn,cp_prod,ventas,cp_superdida,superdida,cp_sequia,cp_helada,cp_plagas,cp_enfermedades,cp_inundacion,cp_otra,cp_edad,edad1,edad2,edad3,cp_vcomún,cp_vmejorada,cp_vhibrinacio,cp_vhibrinterna,cp_sriego,cp_sufertilizantes,cp_sufitosanitario,nuevacondicion,fact_exp_fin_cultivo
0,12075404000020009,12,7,54,4,2,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,29.25,8,8.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,...,3.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,411,NaN,2.0,NaN,4,1.636364,1.636364,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1.49,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN,1.49,1,0.760526
1,12065104000040018,12,6,51,4,4,18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,33.00,8,7.0,1.0,2.0,1.0,1.0,2.0,2.0,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,6.818182,6.818182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,6.0,NaN,NaN,6.00,NaN,NaN,NaN,NaN,1,0.187382
2,13035102130930012,13,3,51,2,13093,12,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,18.00,4,4.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.50,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,1,4.497901
3,23015001038300002,23,1,50,1,3830,2,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,140.00,6,6.0,NaN,1.0,1.0,NaN,2.0,2.0,NaN,3.0,3.0,NaN,1.0,...,15.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,411,1.0,NaN,NaN,4,87.272727,87.272727,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,40.00,NaN,NaN,40.00,NaN,NaN,NaN,NaN,40.00,1,0.474716
4,12065104000040019,12,6,51,4,4,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,20.00,5,5.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,9.090909,9.090909,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,9.99,NaN,NaN,NaN,9.99,NaN,NaN,NaN,NaN,1,0.298237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,9105003011090009,9,10,50,3,1109,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.50,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,1.045455,1.045455,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0.50,NaN,NaN,NaN,0.50,NaN,NaN,0.50,0.50,0.50,1,344.908638
2444,9105003011090002,9,10,50,3,1109,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,2.181818,2.181818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,1.00,NaN,NaN,1.00,NaN,NaN,1.00,1.00,1.00,1,195.695101
2445,9105003011090002,9,10,50,3,1109,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,0.818182,0.818182,0.

In [8]:
cacao['Identificador'].duplicated().sum()

316

###MAYOR A 2 AÑOS

In [9]:
mascara = cacao['cp_k406'] > 2
cacao = cacao[mascara]
cacao

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,tratamiento_compra_insumos,tratamiento_ventas_transporte,tratamiento_ventas_demanda,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,...,cp_cantidad_ins_pq,cp_umed_ins_pq,cp_color_ins_pq,cp_cantidad_fun_pq,cp_umed_fun_pq,cp_color_fun_pq,cp_cantidad_pq,cp_umed_pq,cp_color_pq,cp_trabocacont,cp_trabfami,cp_obsotroscp,cp_nclavr,cp_trs,cp_trc,cp_tro,cp_tn,cp_prod,ventas,cp_superdida,superdida,cp_sequia,cp_helada,cp_plagas,cp_enfermedades,cp_inundacion,cp_otra,cp_edad,edad1,edad2,edad3,cp_vcomún,cp_vmejorada,cp_vhibrinacio,cp_vhibrinterna,cp_sriego,cp_sufertilizantes,cp_sufitosanitario,nuevacondicion,fact_exp_fin_cultivo
0,12075404000020009,12,7,54,4,2,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,29.25,8,8.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,...,3.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,411,NaN,2.0,NaN,4,1.636364,1.636364,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1.49,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN,1.49,1,0.760526
1,12065104000040018,12,6,51,4,4,18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,33.00,8,7.0,1.0,2.0,1.0,1.0,2.0,2.0,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,6.818182,6.818182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,6.0,NaN,NaN,6.00,NaN,NaN,NaN,NaN,1,0.187382
3,23015001038300002,23,1,50,1,3830,2,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,140.00,6,6.0,NaN,1.0,1.0,NaN,2.0,2.0,NaN,3.0,3.0,NaN,1.0,...,15.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,411,1.0,NaN,NaN,4,87.272727,87.272727,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,40.00,NaN,NaN,40.00,NaN,NaN,NaN,NaN,40.00,1,0.474716
4,12065104000040019,12,6,51,4,4,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,20.00,5,5.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,9.090909,9.090909,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,9.99,NaN,NaN,NaN,9.99,NaN,NaN,NaN,NaN,1,0.298237
5,6105002000090003,6,10,50,2,9,3,1,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,212.00,1,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,411,1.0,NaN,NaN,4,9.090909,9.090909,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,5,28.00,NaN,NaN,NaN,28.00,NaN,NaN,NaN,28.00,28.00,0,0.813630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,9105003011090009,9,10,50,3,1109,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.50,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,1.045455,1.045455,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0.50,NaN,NaN,NaN,0.50,NaN,NaN,0.50,0.50,0.50,1,344.908638
2444,9105003011090002,9,10,50,3,1109,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,2.181818,2.181818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,1.00,NaN,NaN,1.00,NaN,NaN,1.00,1.00,1.00,1,195.695101
2445,9105003011090002,9,10,50,3,1109,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,0.818182,

###MUESTRA PARA PRUEBAS

In [10]:
muestra = rd.sample(list(cacao['Identificador'][cacao['Identificador'].duplicated()]), 5)
muestra

[9095002129140004,
 13045303014940026,
 22035002003150004,
 2075002001980009,
 5035104000950006]

###TOMAR EL PRIMER DATO ELIMINANDO DUPLICADOS

In [11]:
cacao_res = cacao.drop_duplicates(subset=['Identificador'], keep='first', ignore_index=True)
cacao_res

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,tratamiento_compra_insumos,tratamiento_ventas_transporte,tratamiento_ventas_demanda,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,...,cp_cantidad_ins_pq,cp_umed_ins_pq,cp_color_ins_pq,cp_cantidad_fun_pq,cp_umed_fun_pq,cp_color_fun_pq,cp_cantidad_pq,cp_umed_pq,cp_color_pq,cp_trabocacont,cp_trabfami,cp_obsotroscp,cp_nclavr,cp_trs,cp_trc,cp_tro,cp_tn,cp_prod,ventas,cp_superdida,superdida,cp_sequia,cp_helada,cp_plagas,cp_enfermedades,cp_inundacion,cp_otra,cp_edad,edad1,edad2,edad3,cp_vcomún,cp_vmejorada,cp_vhibrinacio,cp_vhibrinterna,cp_sriego,cp_sufertilizantes,cp_sufitosanitario,nuevacondicion,fact_exp_fin_cultivo
0,12075404000020009,12,7,54,4,2,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,29.25,8,8.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,...,3.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,411,NaN,2.0,NaN,4,1.636364,1.636364,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1.49,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN,1.49,1,0.760526
1,12065104000040018,12,6,51,4,4,18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,33.00,8,7.0,1.0,2.0,1.0,1.0,2.0,2.0,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,6.818182,6.818182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,6.0,NaN,NaN,6.00,NaN,NaN,NaN,NaN,1,0.187382
2,23015001038300002,23,1,50,1,3830,2,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,140.00,6,6.0,NaN,1.0,1.0,NaN,2.0,2.0,NaN,3.0,3.0,NaN,1.0,...,15.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,411,1.0,NaN,NaN,4,87.272727,87.272727,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,40.00,NaN,NaN,40.0,NaN,NaN,NaN,NaN,40.00,1,0.474716
3,12065104000040019,12,6,51,4,4,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,20.00,5,5.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,411,NaN,2.0,NaN,4,9.090909,9.090909,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,9.99,NaN,NaN,NaN,9.99,NaN,NaN,NaN,NaN,1,0.298237
4,6105002000090003,6,10,50,2,9,3,1,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,212.00,1,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,411,1.0,NaN,NaN,4,9.090909,9.090909,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,5,28.00,NaN,NaN,NaN,28.0,NaN,NaN,NaN,28.0,28.00,0,0.813630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,9105003011090001,9,10,50,3,1109,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,0.909091,0.909091,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,0.80,NaN,0.80,NaN,NaN,NaN,0.8,0.8,0.80,1,196.661533
1913,9105003011090009,9,10,50,3,1109,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.50,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,1.045455,1.045455,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0.50,NaN,NaN,NaN,0.5,NaN,NaN,0.5,0.5,0.50,1,344.908638
1914,9105003011090002,9,10,50,3,1109,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.42,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,411,NaN,2.0,NaN,4,2.181818,2.181818

### CRITERIO HECTAREA MAYOR

In [12]:
def criterio_hectarea_mayor(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1 and df_repetidos[mascara][columna].nunique() > 1:
      array = df_repetidos[mascara]['cp_k409'].values
      array = np.nan_to_num(array, nan=-np.inf)
      pos_max = array.argmax()
      valor = df_repetidos[mascara][columna].values[pos_max]
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [13]:
# [x for x in cacao_res.columns if x.startswith('cp')]

In [14]:
cacao_res = criterio_hectarea_mayor(cacao, cacao_res, 'ual_parr')
cacao_res = criterio_hectarea_mayor(cacao, cacao_res, 'cp_afecta_prod')

In [15]:
# arroz[arroz['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']].sort_values('Identificador')
cacao[cacao['Identificador'] == 12055002029470003][['Identificador', 'ual_parr', 'cp_k409', 'cp_afecta_prod', 'cp_riego', 'cp_numl', 'cp_numt']]

,Identificador,ual_parr,cp_k409,cp_afecta_prod,cp_riego,cp_numl,cp_numt
1167,12055002029470003,50,1.0,7.0,0,1,1
1168,12055002029470003,50,2.5,7.0,0,2,2


In [16]:
# arroz_res[arroz_res['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]
cacao_res[cacao_res['Identificador'] == 12055002029470003][['Identificador', 'ual_parr', 'cp_k409', 'cp_afecta_prod', 'cp_riego', 'cp_numl', 'cp_numt']]

,Identificador,ual_parr,cp_k409,cp_afecta_prod,cp_riego,cp_numl,cp_numt
946,12055002029470003,50,1.0,7.0,0,1,1


### CRITERIO SUMA

In [17]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      suma  = array.sum()
      df_unicos.loc[id, columna] = suma
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [18]:
cacao_res = criterio_suma_valores(cacao, cacao_res, 'cp_k409')
cacao_res = criterio_suma_valores(cacao, cacao_res, 'ventas')
cacao_res = criterio_suma_valores(cacao, cacao_res, 'cp_cantidad_npk_fq')

In [19]:
cacao[cacao['Identificador'] == 12055002029470003][['Identificador', 'cp_k409', 'ventas', 'cp_cantidad_npk_fq']]

,Identificador,cp_k409,ventas,cp_cantidad_npk_fq
1167,12055002029470003,1.0,0.681818,NaN
1168,12055002029470003,2.5,1.590909,NaN


In [20]:
cacao_res[cacao_res['Identificador'] == 12055002029470003][['Identificador', 'cp_k409', 'ventas', 'cp_cantidad_npk_fq']]

,Identificador,cp_k409,ventas,cp_cantidad_npk_fq
946,12055002029470003,3.5,2.272727,0.0


### CRITERIO BASTA QUE HAYA UN UNO PARA QUE TODOS SEAN UNO

In [21]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      booleano = np.any(array)
      valor = int(booleano)
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [22]:
columnas = ['pa_arealm', 'eq_emotoguadana',	'eq_ebmanual']
for columna in columnas:
  cacao_res = criterio_suma_valores(cacao, cacao_res, columna)

In [23]:
cacao[cacao['Identificador'] == 12055002029470003][['Identificador'] + columnas]

,Identificador,pa_arealm,eq_emotoguadana,eq_ebmanual
1167,12055002029470003,0.0,0.0,1.0
1168,12055002029470003,0.0,0.0,1.0


In [24]:
cacao_res[cacao_res['Identificador'] == 12055002029470003][['Identificador'] + columnas]

,Identificador,pa_arealm,eq_emotoguadana,eq_ebmanual
946,12055002029470003,0.0,0.0,1.0


<hr>

In [25]:
cacao[cacao['Identificador'] == 12055002029470003][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
1167,12055002029470003,3.5
1168,12055002029470003,3.5


In [26]:
cacao_res[cacao_res['Identificador'] == 12055002029470003][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
946,12055002029470003,3.5


### GUARDAR CAMBIOS

In [27]:
cacao_res['Identificador'] = cacao_res['Identificador'].astype('str')
cacao_res.to_csv(os.path.join(ruta_resultados, 'cacao.csv'), index=False, encoding='utf-8')

In [28]:
cacao_res.to_stata(os.path.join(ruta_resultados, 'cacao.dta'), write_index=False)